In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bartpy.sklearnmodel import SklearnModel

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

In [ ]:
def naive_roc_auc_score(y_true, y_pred):
  
  y_true = y_true.to_numpy()
  
  num_same_sign = 0
  num_pairs = 0
  
  for a in range(len(y_true)):
    for b in range(len(y_true)):
      if y_true[a] > y_true[b]:
        num_pairs += 1
        if y_pred[a] > y_pred[b]:
          num_same_sign += 1
        elif y_pred[a] == y_pred[b]:
          num_same_sign += .5
        
  return num_same_sign / num_pairs

score = make_scorer(naive_roc_auc_score, greater_is_better=True)

In [ ]:
data = pd.read_csv('Full_Descriptors.csv')
data = data.dropna()
y = data['LUMO']
X = data.loc[:, ['Molecular Weight', 'Heavy Atom Molecular Weight', 'Max Absolute Partial Charge', 'Max Partial Charge', 
         'Min Abs Partial Charge', 'Min Partial Charge', 'Radical Electrons', 'Valence Electrons', 'NHOH Count', 'NO Count',
         'H Acceptors', 'H Donors', 'Ring Count', 'Aliphatic Rings', 'Aromatic Rings', 'Saturated Rings', 'Aromatic Carbocycles',
         'Aromatic Heterocycles', 'Heteroatoms', 'Rotatable Bonds', 'Saturated Carbocycles', 'Saturated Heterocycles', 'H Count',
         'C Count', 'N Count', 'F Count', 'Halogen Count', 'Double Bonds', 'Triple Bonds']]
list_numerical = X.columns


# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
model = SklearnModel(n_burn=50, n_chains=1, n_jobs=1, n_samples=50, n_trees=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
# scaler = StandardScaler().fit(X_train[list_numerical])
# X_train[list_numerical] = scaler.transform(X_train[list_numerical])
# X_test[list_numerical] = scaler.transform(X_test[list_numerical])

model.fit(X_train, y_train)
# plt.scatter(y_test, model.predict(X_test))
# plt.scatter(X_test, y_test)

In [ ]:
# Use the forest's predict method on the test data
predictions = model.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
from sklearn.metrics import mean_squared_error

prediction = model.predict(X_test)
mse = mean_squared_error(y_test, prediction)
#calculate R-squared of regression model
r_squared = model.score(X_test, y_test)

#view R-squared value
print("R2: ", r_squared)
rmse = mse**.5
print("MSE: ", mse)
print("RMSE: ", rmse)

In [ ]:
param_grid = {
    "n_burn": [0,10,50,100],
    "n_chains": [0,1,3,5,10,15,30],
    "n_trees": [2,5,7,10,20],
}

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = SklearnModel(n_burn=50, n_chains=1, n_jobs=1, n_samples=50, n_trees=10)
grid_cv = GridSearchCV(model, param_grid, scoring=score, n_jobs=-1, cv=cv).fit(X_train, y_train)

In [ ]:
model = SklearnModel(n_burn=50, n_chains=1, n_jobs=1, n_samples=50, n_trees=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
# scaler = StandardScaler().fit(X_train[list_numerical])
# X_train[list_numerical] = scaler.transform(X_train[list_numerical])
# X_test[list_numerical] = scaler.transform(X_test[list_numerical])

model.fit(X_train, y_train)

model.prediction_samples

# sorted_idx = model.feature_importances_.argsort()
# plt.barh(list_numerical[sorted_idx], rfr.feature_importances_[sorted_idx])
# plt.xlabel("Random Forest Feature Importance")
# plt.title("LUMO")